# A tale of Pymoli  [Pythonizing it!]


### Describe the given data to obtain information that can be use to make decisions.

In [1]:
#Importing modules
import pandas as pd
from IPython.display import display, HTML
import statistics
from babel.numbers import format_currency

#assign the data to a panda object
game=pd.read_csv("heroes.csv")

#------- Format for the floats for this work
pd.options.display.float_format = '${:,.2f}'.format

#-------let's see if the data was imported correctly
#-------printing a portion of the data
game.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


## Player Count

#### And... how many players do the land of Pymoli have?

In [2]:
#--------assigning to a variable the count of the unique values of SN
count=len(game['SN'].unique())

#-------creating a DF to present the information
c={'Number of players':[count]}
gamecount=pd.DataFrame(data=c)
display(gamecount)


,Number of players
0,576


 ## Purchasing Analysis (Total)

In [3]:
#-------variables to calculatea average, total number of purchases
#-------revenue and unique items

items=len(game['Item ID'].unique())

average = game['Price'].mean()
total_purchases=game['Purchase ID'].count()
revenue=format_currency(game['Price'].sum(),'USD', locale='en_US')

#--- Formatting data
#creatind a DF wit the information obtained it
#visualization purpose

r={'Number Unique items':[items], 'Average Price':[average], 'Number of Purchases':[total_purchases],
       'Total Revenue':[revenue]}
resume=pd.DataFrame(data=r)
resume = resume[['Number Unique items','Average Price','Number of Purchases','Total Revenue']]
display(resume)

,Number Unique items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


##  Gender Demographics

In [4]:

#it is needed to quit the duplicated accounts in order to have clean data 
#and know how many users are in total
g=game.drop_duplicates('SN', keep = 'last',inplace=False)

#the gender is grouped and counted
gender= pd.DataFrame(g.groupby('Gender')['SN'].count()).sort_values(by='SN', ascending= False)
#the percentage for each gender it is calculated and assigned to a new column
gender['Percentage of players']=gender['SN']/gender['SN'].sum()

#------- Formatting the data--------

gender.rename(columns={'SN': 'Total Count'}, inplace= True)
gender_demo=gender.style.format({'Percentage of players':'{:.2%}'})

display(gender_demo)

,Total Count,Percentage of players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


##  Purchasing Analysis (Gender)

In [5]:
#------- Subsettng the data to perform the required operations: 
#------- * Counting the users by gender and assigned it to a DF
#------- * Creating three new columns for the following operations:
#-------------- *The average of purchases by gender
#-------------- *How many items are bought by gender
#-------------- *And, determine the average pruchase by person (unique users)


purchasing_gender= pd.DataFrame(game.groupby('Gender')['Price'].sum())
purchasing_gender['Average Purchase Price'] =(game.groupby('Gender')['Price'].mean())
purchasing_gender['Purchase Count'] = (game.groupby('Gender')['SN'].count())
purchasing_gender['Avg Total Purchase per Person']=(game.groupby('Gender')['Price'].sum())/(game.groupby('Gender')['SN'].nunique())

#------- Formatting -------
purchasing_gender.rename(columns={'Price':'Total Purchase Value'}, inplace= True)
purchasing_gender = purchasing_gender[['Purchase Count','Average Purchase Price','Total Purchase Value','Avg Total Purchase per Person']]
display(purchasing_gender)

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

In [6]:
#The bins are created
bins=[4,9,14,19,24,29,34,39,45]
#The labels for the bins are specified
labels_age=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
#It is ceated a column to place the bins
game['Bins']=pd.cut(game['Age'],bins, labels = labels_age,include_lowest=True )
#A DF without duplicates is created
game_dup=game.drop_duplicates('SN')
#It is created a DF grouped by Bins and counting how mane user per age are.
game_age=pd.DataFrame(game_dup.groupby('Bins')['Age'].count())
#Rename the columns
game_age.columns={'Total Count'}
#A new column is created, where the percentage of players is going to be placed
game_age['Percentage of Players']=game_age['Total Count']/game_age['Total Count'].sum()

#------ Formatting ------
game_age = game_age.style.format({'Percentage of Players':'{:.2%}'})

display(game_age)



,Total Count,Percentage of Players
Bins,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

In [7]:
#------- creating the bins for the data
bins=[4,9,14,19,24,29,34,39,45]

#------- creating the labes for the bins
labels_age=['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']

#-------- Creating a copy of the original DF
p_a = game.copy()
#-------- Binning the data
p_a['Age Ranges']=pd.cut(p_a['Age'],bins, labels = labels_age,include_lowest=True )
#-------- Making the maths:
#-------- * Creating a DF with the total of users by age
#-------- * Average of consuming by age
#-------- * Total purchase by age
#-------- * Counting the total of users by age without duplicates
#-------- * Average of consuming by person

age_pur_analysis = pd.DataFrame(p_a.groupby('Age Ranges')['Age'].count())
age_pur_analysis['Average Purchase Price']=pd.DataFrame(p_a.groupby(['Age Ranges'])['Price'].mean())
age_pur_analysis['Total Purchase Value']=pd.DataFrame(p_a.groupby(['Age Ranges'])['Price'].sum())
u_nique=pd.DataFrame(p_a.groupby('Age Ranges')['SN'].nunique())
age_pur_analysis['Avg Total Purchase per Person'] = age_pur_analysis['Total Purchase Value']/u_nique['SN']

#------ Formatting------
age_pur_analysis.rename(columns={'Age':'Purchase Count'}, inplace=True)

display(age_pur_analysis)


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


##  Top Spenders

In [8]:
#-------- Creating a copy of the original DF
spenders=game.copy()
#------- Group the data by user and the total pruchase 
buyers_users = pd.DataFrame(spenders.groupby('SN')['Gender'].count()).sort_values(by=['Gender'],ascending = False ).reset_index()
buyers_purchase_av = pd.DataFrame(spenders.groupby('SN')['Price'].sum()).sort_values(by= ['Price'], ascending = False).reset_index()

buyers=(buyers_users.merge(buyers_purchase_av, on = 'SN')).sort_values(by=['Price'], ascending = False)
buyers['Average Purchase Price'] = buyers['Price']/buyers['Gender']

#-------- Formatting-------


buyers=buyers.rename(columns={'Gender':'Purchase Count',
                     'Price':'Total Purchase Value'})
buyers = buyers[['SN','Purchase Count','Average Purchase Price','Total Purchase Value']]
buyers.set_index('SN', inplace= True)
buyers.head()


,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

In [28]:
#------- Copy the original DF
popular_items = game.copy()

#------ Goruping tha data by Item ID and Name, and counting the users:
#------ Adding a column for the total purchase
#------ and a column for the price per item
items= pd.DataFrame(popular_items.groupby(['Item ID','Item Name'])['SN']
                                .count()).sort_values(by='SN',ascending=False)
items['Total Purchase Value']=pd.DataFrame(popular_items.groupby(['Item ID', 'Item Name'])['Price'].sum())
items['Item Price'] = items['Total Purchase Value']/items['SN']
items.rename(columns={'SN':'Purchase Count'}, inplace=True)
items = items[['Purchase Count','Item Price', 'Total Purchase Value']]

popular_items.head()
display(items.head())

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

In [29]:
#sorting the data by Total Purchase value
most_profitable = items.sort_values(by='Total Purchase Value',ascending=False)

most_profitable.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
